In [5]:
import numpy as np
from itertools import combinations
from itertools import permutations
from scipy import stats
from urllib import request
import re
import pandas as pd

# Введение

В данном ТЗ описывается инструмент автоматического поиска популярных тегов и пометки ими запросов из загруженного ядра. Основной задачей инструмента является подготовка файла для оценки видимости. Кроме того, приведён примерный код, реализующий данный функционал. Код написан для того чтобы протестировать идею сервиса и не является образцом хорошего програмирования.

Сервис разбивает ключевые слова на 1-2 и 3х словные сочетания, находит устойчивые  двусловные и трехсловные сочетания, после чего помечает запросы тегми - однословными + устойчивыми двусловными и трёхсловными сочетаниями

# Категории

Весь описаный ниже процесс выполняется отдельно для каждой категории. В данном примере во входном файле только одна категория, но вам необходимо дать пользователю возможность загрузить список запросов для многих категорий, помеченных названиями категорий в формате:

ключевое слово 1 ; категория 1

ключевое слово 2 ; категория 1

ключевое слово 3 ; категория 2

и так далее.

# Лемматизируем и чистим запросы

В терминале используя майтем:  $mystem -c -l -d input.txt output.txt

Список лемматизированных запросов далее используется для поиска и привязки тегов, но пользователю сервиса нужно будет отдать список запросов точно в такой-же форме, в какой они были получены из базы.

In [51]:
#Загрузка файла с запросами, где все слова приведены к исходной словоформе.
qfile = open('../divanylem.txt', 'r', encoding = "utf-8").read().split('\n')
#qfile = open('../platyalem.txt', 'r', encoding = "utf-8").read().split('\n')

In [52]:
qfile[2:7]

['2 {х} {местный} {диван} {кровать}',
 '31 {диван} {для} {офис}',
 '32 {диван} {офисный}',
 '33 {купить} {диван}',
 '35 {диван} {еврокнижка?}']

In [53]:
#Удаление ненужных символов
def clean_queries(qlist, regexp_delete, regexp_use = ''):
    for i in range(len(qlist)):
        qlist[i] = re.sub(regexp_delete, regexp_use, qlist[i])

clean_queries(qfile, '[?{}}]', "")

In [54]:
qfile[2:7]

['2 х местный диван кровать',
 '31 диван для офис',
 '32 диван офисный',
 '33 купить диван',
 '35 диван еврокнижка']

# Подсчёт встречаемости слов и 2-3-словных сочетаний

По полученному списку запросов, необходимо посчитать сколько раз встречаются отдельные слова и 2-3 словные сочетания. Будем считать, что слова являются сочетанием, если они находятся в одном запросе, не зависимо от порядка и близости из относительно друг-друга. Но за то, что слова идут друг за другом, можно дать им дополнительный балл. Например, считать что если 2 слова встретились в одном запросе, то пара слов получает 1 балл, а если встретились подряд - 2 балла. Порядок учитывать не нужно.


In [55]:
def count_ngrams(keywords, smallest, comb_size=1, bonus = 2,
                stop_words = []):
    """
    keywords - список ключевиков
    smallest - частотность самого редкого сочетания/слова на выходе
    comb_size - 1 для того чтобы посчитать однословники, 2 - двусловники и так далее.
    bonus - сколько баллов получает словосочетание, если его слова идут друг за другом в запросе.
    stop_words - слова, не включаемые в рассчёт.
    """
    if comb_size == 1:
        #список униграм
        index = {}
        for line in keywords:
            for word in line.split():
                if word in index:
                    index[word] +=1
                else:
                    if word not in stop_words:
                        index[word] = 1  
    else:
        index = {}
        for line in keywords:
            #Каждая ключевая фраза разбивается на слова и этот список слов сортируется, для того чтобы избежать повторений
            #и считать сочетания слов в разном порядке как одно.
            line_list = line.split()
            line_list.sort()
            for comb in combinations(line_list, comb_size):
                if comb in index:
                    #Важный момент! при проверке не идут ли слова подряд, используются все возможные варианты порядка слов
                    perms = [' '.join(perm) for perm in permutations(comb, comb_size)]
                    j = 0
                    for perm in perms:
                        if perm in line:
                            index[comb] += bonus
                            j == 1
                            break
                    if j == 0:
                        index[comb] +=1
                else:
                    s = 0
                    for word in comb:
                        if word in stop_words:
                            s += 1
                    if s == 0:
                        index[comb] = 1
    
    #удаление слов/словосочетаний которые встречаются реже чем smallest
    index_clean = {}
    for ngram in index:
        if index[ngram] >= smallest:
            index_clean[ngram] = index[ngram]
   
    return index_clean


In [56]:
stop_words = ['с','в','на','от','к','за','до','под','над','со','и', 'из', 'по', 'а', 'где', 'для']

ngrams1 = count_ngrams(qfile, smallest= 1, comb_size = 1, bonus = 2, stop_words=stop_words)
ngrams2 = count_ngrams(qfile, smallest = 15, comb_size = 2, bonus = 2, stop_words=stop_words)
ngrams3 = count_ngrams(qfile, smallest = 10, comb_size = 3, bonus = 2, stop_words=stop_words)

Получили списки слов и сочетаний с оценками встречаемости в загруженной семантике

In [57]:
ngrams1

{'03': 1,
 '04': 1,
 '05': 1,
 '06': 1,
 '07': 1,
 '1': 4,
 '10': 1,
 '100': 3,
 '10000': 1,
 '11': 2,
 '12': 1,
 '120': 2,
 '180': 1,
 '2': 18,
 '200': 3,
 '200х200': 1,
 '25': 1,
 '26': 1,
 '2х': 1,
 '3': 13,
 '300': 2,
 '31': 1,
 '32': 1,
 '33': 1,
 '35': 1,
 '36': 1,
 '37': 1,
 '38': 1,
 '39': 1,
 '3х': 1,
 '4': 2,
 '400': 1,
 '5': 1,
 '500': 1,
 '5000': 1,
 '600': 1,
 '68566': 1,
 '7': 1,
 '8': 3,
 '9': 1,
 'a': 5,
 'advance': 1,
 'airon': 1,
 'alecto': 1,
 'alivar': 1,
 'amika': 1,
 'angelo': 1,
 'arketipo': 1,
 'ashley': 3,
 'baker': 1,
 'bangkok': 1,
 'baxter': 3,
 'bedding': 1,
 'bentley': 1,
 'bestway': 1,
 'bm': 1,
 'bo': 8,
 'bodema': 1,
 'bonaldo': 3,
 'box': 8,
 'busnelli': 1,
 'c': 2,
 'cappellini': 1,
 'caspani': 1,
 'chelini': 1,
 'chester': 2,
 'chesterfield': 1,
 'circle': 1,
 'conca': 1,
 'corniche': 1,
 'country': 1,
 'cousins': 2,
 'cts': 1,
 'cube': 1,
 'de': 1,
 'domingo': 2,
 'duresta': 2,
 'ego': 1,
 'eichholtz': 1,
 'ella': 1,
 'eva': 1,
 'evanty': 1,
 'evolu

In [58]:
ngrams2

{('2', 'диван'): 28,
 ('2', 'местный'): 15,
 ('2', 'х'): 16,
 ('3', 'диван'): 21,
 ('3', 'х'): 16,
 ('bo', 'box'): 22,
 ('mobel', 'zeit'): 16,
 ('v', 'диван'): 16,
 ('аккордеон', 'диван'): 227,
 ('аккордеон', 'купить'): 23,
 ('аккордеон', 'недорого'): 18,
 ('амстердам', 'диван'): 42,
 ('английский', 'диван'): 15,
 ('бар', 'диван'): 21,
 ('без', 'диван'): 42,
 ('без', 'подлокотник'): 52,
 ('белоруссия', 'диван'): 34,
 ('белорусский', 'диван'): 83,
 ('белый', 'диван'): 83,
 ('белый', 'кожаный'): 24,
 ('белый', 'черно'): 19,
 ('блок', 'диван'): 17,
 ('блок', 'пружинный'): 40,
 ('большой', 'диван'): 70,
 ('большой', 'угловой'): 15,
 ('борович', 'диван'): 36,
 ('борович', 'мебель'): 19,
 ('бостон', 'диван'): 18,
 ('бристоль', 'диван'): 34,
 ('бруклин', 'диван'): 19,
 ('бу', 'диван'): 21,
 ('выкатной', 'диван'): 139,
 ('выкатной', 'купить'): 37,
 ('германия', 'диван'): 26,
 ('гостиная', 'диван'): 34,
 ('двухместный', 'диван'): 80,
 ('двухместный', 'купить'): 20,
 ('двухместный', 'офисный'): 

# Поиск устойчивых словосочетаний

Основная задача сервиса - получив семантику от оптимизатора, правилно выделить из неё теги и пометить ими запросы. Сложность состоит в том, что не все теги являются однословными, часто смысл тега теряется если разбить его на слова(например, "Roberto Cavalli" или "индивидуальный пошив"). Соответственно, необходимо найти подобные сочетания и считать их одним тегом.

Оценка того, на сколько словосочетание является устойчивым основывается на частоте употребления отдельных слов из сочетания в запросах и частоте употребления всех слов из сочетания. Чем чаще слова идут вместе в одном запросе, тем выше вероятность что они составляют устойчивое сочетание и его нужно использовать как отдельный тег.

Посчитать то, на сколько сочетание является устойчивым, можно используя гипергеометрическое распределение. <a href = 'http://ru.convdocs.org/pars_docs/refs/115/114050/114050_html_m6184011e.gif'>(формула)</a> для f = P(X = m), где:

N - число ключевых слов для категории умноженное на бонус за то что слова встречаются вместе
M - сколько раз всречается первое слово из анализируемой пары
n - сколько раз встречается второе слово из анализируемой пары
m или x - сколько раз встречается пара слов умножить на 1/(число ключевых слов для категории слов).

используя все эти параметры, нужно найти значение CDF(cumulative distribution function) = F(m, n, M, N) = P(X <= m) гипергеометрического распределения для каждого тега с числом слов > 1.

В итоге, для каждой пары слов получим оценку её "устойчивости". Самые устойчивые пары слов будем считать двусловными тегами. Для оценки словосочетаний из более чем двух слов, считается оценка для каждой пары слов из многословного сочетания и берётся среднее.

Ниже реализация с использованием функции stats.hypergeom.cdf из модуля scipy.

In [59]:
#вероятности hypergeometric
def count_probs(indexn, index, comb_size, nkwords, bonus = 2):
    """
    indexn - список словосочетаний с оценками их встречаемости в запросах категории
    index - список отдельных слов с оценками их встречаемости
    comb_size - размер словосочетаний из indexn в словах
    nkwords - число ключевых слов в категории
    bonus - какой бонус за то что слова идут подряд использовался при оценке встречаемости слов
    
    возвращает список (словарь) {(словосочетание) : [сколько раз слово встречается, оценка устойчивости словосочетания]}
    """
    probs = {}
    #Для каждого словосочетания
    for ngram in indexn:
        if indexn[ngram] > 2:
            probs[ngram] = [indexn[ngram], 0]
            
            #все возможные комбинации чисел от 0 до comb_size, размера 2 (для того чтобы далее с помошью этих индексов
            #пройтись по всем возможным комбинациям из 2х слов в многословном сочетании)
            index_combs = [i for i in combinations(range(comb_size), 2)]
            for i, j in index_combs:
                cdf = stats.hypergeom.cdf(indexn[ngram]*(1/nkwords), nkwords*bonus, index[ngram[i]], index[ngram[j]])
                probs[ngram][1] += cdf
            probs[ngram][1] = probs[ngram][1]/len(index_combs)
            
    return probs

In [60]:
probs2 = count_probs(ngrams2, ngrams1, 2, len(qfile), bonus = 2)
probs3 = count_probs(ngrams3, ngrams1, 3, len(qfile), bonus = 2)

In [61]:
probs2

{('2', 'диван'): [28, 3.7631750768535268e-06],
 ('2', 'местный'): [15, 0.95701310010030793],
 ('2', 'х'): [16, 0.97317905676958527],
 ('3', 'диван'): [21, 0.0001213351855131848],
 ('3', 'х'): [16, 0.98056191743790289],
 ('bo', 'box'): [22, 0.99259819395166526],
 ('mobel', 'zeit'): [16, 0.99583068499031513],
 ('v', 'диван'): [16, 0.015619557018940742],
 ('аккордеон', 'диван'): [227, 2.8844639380241266e-25],
 ('аккордеон', 'купить'): [23, 0.0002669031894416435],
 ('аккордеон', 'недорого'): [18, 0.081948241459197102],
 ('амстердам', 'диван'): [42, 1.5103555623434969e-05],
 ('английский', 'диван'): [15, 0.0078061527999147354],
 ('бар', 'диван'): [21, 1.8780885378820426e-06],
 ('без', 'диван'): [42, 9.3718866595322713e-07],
 ('без', 'подлокотник'): [52, 0.92609354728720406],
 ('белоруссия', 'диван'): [34, 1.443536270822553e-08],
 ('белорусский', 'диван'): [83, 4.443223667367826e-10],
 ('белый', 'диван'): [83, 2.7377810802818081e-11],
 ('белый', 'кожаный'): [24, 0.42291579549908292],
 ('белы

In [62]:
probs3

{('2', 'диван', 'местный'): [13, 0.31900577696305565],
 ('2', 'диван', 'х'): [12, 0.32443471837672494],
 ('2', 'местный', 'х'): [16, 0.96632205396227411],
 ('3', 'диван', 'местный'): [10, 0.32296526927281477],
 ('3', 'диван', 'офисный'): [10, 0.23127644147509274],
 ('3', 'диван', 'х'): [12, 0.32693486260297622],
 ('3', 'местный', 'х'): [16, 0.97270330915799363],
 ('3', 'офисный', 'х'): [10, 0.78932596530529586],
 ('bo', 'box', 'диван'): [16, 0.33346660539752682],
 ('mobel', 'zeit', 'диван'): [16, 0.34235659967606558],
 ('аккордеон', 'дешево', 'диван'): [10, 0.15610474421721487],
 ('аккордеон', 'диван', 'купить'): [39, 8.8967729813881167e-05],
 ('аккордеон', 'диван', 'москва'): [11, 0.014331186801033617],
 ('аккордеон', 'диван', 'недорого'): [18, 0.027316080486399035],
 ('аккордеон', 'диван', 'недорогой'): [13, 0.18527408983786617],
 ('аккордеон', 'диван', 'угловой'): [11, 0.001452910942383971],
 ('без', 'диван', 'подлокотник'): [38, 0.3086981615287297],
 ('белоруссия', 'диван', 'произв

In [65]:
#Можно сразу удалить словосочетания с оценкой встречаемости ниже 0.75
def cut_low_pr(probs, cut = 0.75):
    probs_clean = {}
    for ngram in probs:
        if probs[ngram][1] > cut:
            probs_clean[ngram] = probs[ngram]
    return probs_clean

probs2_high = cut_low_pr(probs2)
probs3_high = cut_low_pr(probs3)

In [66]:
#отсортированные по устойчивости словосочетания
sorted(probs3_high.items(), key=lambda x:x[1][1], reverse=True)

[(('3', 'местный', 'х'), [16, 0.97270330915799363]),
 (('2', 'местный', 'х'), [16, 0.96632205396227411]),
 (('матрас', 'ортопедический', 'тонкий'), [13, 0.8962795088780986]),
 (('3', 'офисный', 'х'), [10, 0.78932596530529586])]

In [67]:
#отсортированные по устойчивости словосочетания
sorted(probs2_high.items(), key=lambda x:x[1][1], reverse=True)

[(('сколько', 'стоить'), [16, 0.99583068499031513]),
 (('mobel', 'zeit'), [16, 0.99583068499031513]),
 (('так', 'тик'), [19, 0.99432871836737091]),
 (('bo', 'box'), [22, 0.99259819395166526]),
 (('раскладушка', 'французский'), [16, 0.9875376620876124]),
 (('3', 'х'), [16, 0.98056191743790289]),
 (('нижний', 'новгород'), [37, 0.97612602173061647]),
 (('2', 'х'), [16, 0.97317905676958527]),
 (('местный', 'х'), [37, 0.96877400501692912]),
 (('белый', 'черно'), [19, 0.96796978950668222]),
 (('2', 'местный'), [15, 0.95701310010030793]),
 (('класс', 'эконом'), [40, 0.95179077982184113]),
 (('деревянный', 'подлокотник'), [22, 0.95133721255104375]),
 (('блок', 'пружинный'), [40, 0.94570885390661785]),
 (('матрас', 'тонкий'), [25, 0.94282137561393964]),
 (('классический', 'стиль'), [19, 0.94241523373593539]),
 (('пошив', 'чехол'), [28, 0.92994865784902858]),
 (('без', 'подлокотник'), [52, 0.92609354728720406]),
 (('петербург', 'санкт'), [64, 0.92136090194020426]),
 (('комната', 'маленький'), [2

# Разметка запросов тегами

После того как были определены популярные двусловные и трехсловные теги, необходимо для каждого запроса обозначить список тегов к которым он принадлежит.

Для этого просто определяем встречаются ли все слова из тега в запросе (включая однословные теги). 

Если к запросу привязаны однословные теги, входящие в состав многословных тегов, составляют устойчивое сочетание которое тоже привязано к запросу, необходимо удалить однословные теги и оставить только много словные. Такой же принцип распространяется к двусловным тегам по отношению к трёхсловным, удаляем двусловники которые есть в составе треёсловного тега привязанного к запросу.

Если к запросу возможно привзать более одного многословного тега, необходимо это сделать, даже если одно слово из запроса встречается в нескольких многословных тегах и не понятно к какому из них оно относится.

In [68]:
def tag(query, tags):
    L = [] 
    if type(tags[0]) is str:
        for word in query.split():
            if word in tags:
                L.append(word)
   
    elif len(tags[0]) == 2:
        terms = query.split()
        terms.sort()
        for comb in combinations(terms, 2):
            if comb in tags:
                L.append(comb)
    
    elif len(tags[0]) == 3:
        terms = query.split()
        terms.sort()
        for comb in combinations(terms, 3):
            if comb in tags:
                L.append(comb)
    return L
    

def tag_queries(queries, tags1w, tags2w, tags3w):
    tagged = {}
    for query in queries:
        qtags1, qtags1_iter = tag(query, tags1w), tag(query, tags1w)
        qtags2, qtags2_iter = tag(query, tags2w), tag(query, tags2w)
        qtags3, qtags3_iter = tag(query, tags3w), tag(query, tags3w)
        for tag1 in qtags1_iter:
            for tag2 in qtags2:
                if tag1 in tag2:
                    try:
                        qtags1.remove(tag1)
                    except:
                        pass
                    break
           
        for tag1 in qtags1_iter:
            for tag3 in qtags3:
                if tag1 in tag3:
                    try:
                        qtags1.remove(tag1)
                    except:
                        pass
                    break
                    
        if len(qtags3) > 0:
            for tag2 in qtags2_iter:
                for tag3 in qtags3:
                    if tag2[0] in tag3 and tag2[1] in tag3:
                        try:
                            qtags2.remove(tag2)
                        except:
                            pass
                        break
        tagged[query] = [qtags1, qtags2, qtags3]            
    return tagged 

In [82]:
ngrams1frequent = {}
for word in ngrams1:
    if ngrams1[word] > 2:
        ngrams1frequent[word] = ngrams1[word]
        

In [83]:
#таблица с ключевиками и тегами
tagged = pd.DataFrame.from_dict(tag_queries(qfile, 
            list(ngrams1frequent.keys()), list(probs2_high.keys()), list(probs3_high.keys())), orient = 'index')

# Результат работы сервиса

(разница только в том, что на выходе нужно ещё привязать к исходным словоформам сами ключевые слова, которые были лемматизированы в сомом начале)

In [85]:
#Важно! Если есть многословный тег, то есдиничных слов из него нет в списке однословных тегов
#и если это тег из 3х слов, то пар слов из него нет в списке двусловных тегов
tagged.iloc[950:1000]

,0,1,2
диван дельфин купить в москва,"[диван, дельфин, купить, москва]",[],[]
диван трансформер купить,"[диван, трансформер, купить]",[],[]
лион диван,"[лион, диван]",[],[]
диван ростов на дон,[диван],[],[]
ортопедический матрас для диван,[диван],"[(матрас, ортопедический)]",[]
палермо диван,"[палермо, диван]",[],[]
диван для бар ресторан,"[диван, бар, ресторан]",[],[]
офисный диван сандра,"[офисный, диван]",[],[]
купить угловой диван от производитель,"[купить, угловой, диван, производитель]",[],[]
диван карпентер,[диван],[],[]
